<!-- ![Two data scientists working on a dashboard.](hr-image-small.png) -->

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

**TODO:**

* Columns containing integers must be stored as 32-bit integers (int32).
* Columns containing floats must be stored as 16-bit floats (float16).
* Columns containing nominal categorical data must be stored as the category data type.
* Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
* The columns of ds_jobs_clean must be in the same order as the original dataset.
* The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [103]:
import pandas as pd

## Data exploration 

In [104]:
ds_jobs = pd.read_csv('customer_train.csv')
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [105]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

## Optimize data frame to reduce memory usage, first 5 points

In [106]:
ds_jobs_clean = ds_jobs.copy()

In [107]:
# Convert int64 columns to int32
int64_columns = ds_jobs_clean.select_dtypes(include=['int64']).columns
ds_jobs_clean[int64_columns] = ds_jobs_clean[int64_columns].astype('int32')

# Convert float64 columns to float16
float64_columns = ds_jobs_clean.select_dtypes(include=['float64']).columns
ds_jobs_clean[float64_columns] = ds_jobs_clean[float64_columns].astype('float16')

# Convert categorical columns to category data type
ds_jobs_clean['city'] = ds_jobs_clean['city'].astype('category')

ds_jobs_clean['gender'] = ds_jobs_clean['gender'].astype('category')

ds_jobs_clean['major_discipline'] = ds_jobs_clean['major_discipline'].astype('category')

ds_jobs_clean['company_size'] = ds_jobs_clean['company_size'].astype('category')

ds_jobs_clean['company_type'] = ds_jobs_clean['company_type'].astype('category')

In [108]:
# Get unique values from ordinal categorical data
print(ds_jobs_clean['experience'].unique())
print(ds_jobs_clean['last_new_job'].unique())
print(ds_jobs_clean['company_size'].unique())
print(ds_jobs_clean['relevant_experience'].unique())
print(ds_jobs_clean['enrolled_university'].unique())
print(ds_jobs_clean['education_level'].unique())

['>20' '15' '5' '<1' '11' '13' '7' '17' '2' '16' '1' '4' '10' '14' '18'
 '19' '12' '3' '6' '9' '8' '20' nan]
['1' '>4' 'never' '4' '3' '2' nan]
[NaN, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10-49', '100-499', '500-999']
Categories (8, object): ['10-49', '100-499', '1000-4999', '10000+', '50-99', '500-999', '5000-9999', '<10']
['Has relevant experience' 'No relevant experience']
['no_enrollment' 'Full time course' nan 'Part time course']
['Graduate' 'Masters' 'High School' nan 'Phd' 'Primary School']


In [109]:
# Convert ordinal categorical columns to ordered categories data type
ds_jobs_clean['experience'] = pd.Categorical(ds_jobs_clean['experience'], categories=['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'], ordered=True)

ds_jobs_clean['last_new_job'] = pd.Categorical(ds_jobs_clean['last_new_job'], categories=['1', '2', '3', '4', '>4'], ordered=True)

ds_jobs_clean['company_size'] = pd.Categorical(ds_jobs_clean['company_size'], categories=['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'], ordered=True)

ds_jobs_clean['relevant_experience'] = pd.Categorical(ds_jobs_clean['relevant_experience'], categories=['No relevant experience', 'Has relevant experience'], ordered=True)

ds_jobs_clean['enrolled_university'] = pd.Categorical(ds_jobs_clean['enrolled_university'], categories=['No_enrollment', 'Part time course', 'Full time course'], ordered=True)

ds_jobs_clean['education_level'] = pd.Categorical(ds_jobs_clean['education_level'], categories=['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)

In [110]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     4955 non-null   category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            16283 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  int32   
dtypes: category(10), float

## Filter to only contain students with 10 or more years of experience at companies with at least 1000 employees, last point

In [111]:
# Filter DataFrame Based on Experience and Company Size
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] > '500-999')]

In [112]:
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,NaN,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,NaN,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,NaN,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0


#### We could see how effective was changing data type in our data frame, reducing memory usage from more than 2 MB to 456.5 KB